<a href="https://colab.research.google.com/github/djangointhehills/Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import copy

In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [3]:
class Perceptron:
    def __init__(self, learning_rate, activation_f):
        self.weights = []
        self.activation_f = activation_f
        self.learning_rate = learning_rate

    def update_weights(self, x, y_hat, y):
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] + self.learning_rate * (y - y_hat) * x[i]

    def train(self, X, Y):
        # Initialise number of weights and add bias.
        self.weights = [0] * len(X[0])
        self.weights.append(0)

        epochs = 0
        train = True
        while train and epochs < 10:
          epochs += 1
          train = False

          # Train over all the X data instances.
          for i in range(len(X)):
              instance = copy.deepcopy(X[i])
              instance.insert(0, 1)
              y_hat = self.activation_f(np.dot(self.weights, instance))
              if y_hat != Y[i]:
                  train = True
                  self.update_weights(instance, y_hat, Y[i])
        print('Training complete!')

    def predict(self, X):
        predictions = []
        for x in X:
          instance = x
          instance.insert(0,1)
          pred = self.activation_f(np.dot(self.weights, instance))
          predictions.append(pred)
        return predictions

In [4]:
model = Perceptron(.3, sigmoid)

In [5]:
X = [[1,2,3,4],
 [3,2,4,4],
 [1,2,12,3]]

Y = [0,0,1]



In [6]:
np.dot([1,2],[3,4])

11

In [7]:
model.train(X,Y)

Training complete!


In [8]:
predictions = model.predict(X)

In [9]:
print(predictions)

[0.0006024306711822066, 0.0002846986661957373, 0.9981355218907151]
